In [1]:
import polars as pl
from docx import Document
from utils.config import get_db_connection


def generate_sample_contract_report(conn, contract_number):
    """Generate a sample report for 100 products that a contract has and returns up to 3
      competitor products for each.

        Args:
            conn (Connection): The database connection.
            contract_number (str): The contract number to generate the report for.

        Returns:
            combined_df (pl.DataFrame): The DataFrame with the combined data.

      """
    query = f"""
    WITH reference_items AS (
        -- Get 100 random items from the specific contract
        SELECT *
        FROM gsa_product_extract_jan2024
        WHERE contract_number =  '{contract_number}'
        ORDER BY RANDOM()  -- Randomize the selection of the 100 items
        LIMIT 100
    ),
    competitor_items AS (
        -- Get all items with the same manufacturer_part_number from different contracts
        SELECT *
        FROM gsa_product_extract_jan2024 gi
        WHERE gi.contract_number != '{contract_number}'  -- Ensure items are from different contracts
        AND gi.manufacturer_part_number IN (SELECT manufacturer_part_number FROM reference_items)
    )
    -- Combine reference items with their matches
    SELECT ref.*, 'reference' AS source
    FROM reference_items ref
    UNION ALL
    SELECT comp.*, 'competitor' AS source
    FROM competitor_items comp
    ORDER BY jprod_id;
    """

    df = pl.read_database(query, conn)

    # Make Correct Data Type so shit doesn't fuck up calculations
    df = df.with_columns(pl.col("price").cast(pl.Float64))

    # Calculate the comp average price for each manufacturer_part_number
    # for competitor products only
    comp_average_price = df.filter(pl.col("source") == "competitor").group_by("manufacturer_part_number").agg(
        pl.col("price").mean().alias("average_price_on_gsa")
    )

    # Calculate the standard deviation of prices for each manufacturer_part_number
    price_deviation = df.group_by("manufacturer_part_number").agg(
        pl.col("price").std().alias("price_deviation")
    )

    # Combine the average price and price deviation into a single DataFrame
    price_comparison = comp_average_price.join(price_deviation, on="manufacturer_part_number", how="left")

    print("PRICE_COMPS_1")
    print("******************************************")
    print(price_comparison)

    # Select the columns to include in the final report from the original contractors items
    selected_columns = df.select([
        "contractor_name",
        "contract_number",
        "manufacturer_part_number",
        "manufacturer_name",
        "product_name",
        "price",
    ]).filter(pl.col("contract_number") == contract_number)

    # Combine the selected columns with the calculated columns
    combined_df = selected_columns.join(price_comparison, on="manufacturer_part_number", how="left")

    # Calculate the percent difference from the Contractors price vs the average price on GSA
    combined_df = combined_df.with_columns(
        ((pl.col("price") - pl.col("average_price_on_gsa")) / pl.col("average_price_on_gsa")).alias("percent_difference")
    )

    return combined_df

In [2]:
connection = get_db_connection()
report_df = generate_sample_contract_report(connection, '47QSEA20D003B')

PRICE_COMPS_1
******************************************
shape: (96, 3)
┌──────────────────────────┬──────────────────────┬─────────────────┐
│ manufacturer_part_number ┆ average_price_on_gsa ┆ price_deviation │
│ ---                      ┆ ---                  ┆ ---             │
│ str                      ┆ f64                  ┆ f64             │
╞══════════════════════════╪══════════════════════╪═════════════════╡
│ TOP25332                 ┆ 19.484762            ┆ 8.079391        │
│ HR-155-2-AL              ┆ 477.976667           ┆ 45.302516       │
│ 12110063-PKG             ┆ 0.66                 ┆ 0.130266        │
│ 5760553                  ┆ 177.458571           ┆ 15.353243       │
│ 1.5510- TRILOCK          ┆ 81.63                ┆ 9.94217         │
│ …                        ┆ …                    ┆ …               │
│ 3911.30                  ┆ 66.856667            ┆ 6.047886        │
│ BBK-035-G                ┆ 25.412               ┆ 35.556318       │
│ 7057745         